In [ ]:
from wordsegment import load, segment
load()

In [ ]:
# Read the text file
with open('../Data/emnlp_dict.txt', 'r') as file:
    lines = file.readlines()

# Initialize an empty dictionary
word_dict = {}

# Loop through pairs and create a dictionary
for pair in lines:
    #remove \n from the end of the line
    pair = pair.strip()
    words = pair.split('\t')  # Split the pair by the tab character
    if len(words) == 2:
        word_dict[words[0]] = words[1]

print(word_dict)

In [ ]:
import pickle as pkl

In [ ]:
#picle the dictionary
with open('../Data/emnlp_dict.pkl', 'wb') as file:
    pkl.dump(word_dict, file)

In [ ]:
#given a string give the same string but all words in lowercase unless all the word is in uppercase
def lower_case(text):
    if text.isupper():
        return text
    else:
        return text.lower()

In [ ]:
lower_case('HELLo')

In [ ]:
word = 'YES HELLoooOOYESSsSIR'
segmented = segment(word)
print(segmented)

In [ ]:

def make_two_consecutive(string):
    return re.sub(r'(.)\1{2,}', r'\1\1', string)

In [ ]:
make_two_consecutive('Yeeeeeeesss suuuuur')

In [ ]:
segmented2 = segment('GOOOOAAAAALLLLLL by Barcelona')
print(segmented2)

In [ ]:
def correct_word(word:str)->str:
    if word in word_dict:
        return word_dict[word]
    else:
        return word

In [ ]:
segmented3= segment('HELLOOOYESSSIR')
segmented3 

In [14]:
def segment_word (text: str)-> list:
    return segment(text)

In [ ]:
segmented4 = segment_word('HELLOOOYESSSIR')
print(segmented4)

In [ ]:

def replace_hashtag(text: str)-> str:
    return re.sub(r'#', ' talking about ', text)

In [12]:
def replace_hashtag(text: str)-> str:
    """
    Splits the text by space and check if word starts with # and if it does it replaces it with talking about
    and segment the word right after the #
    """
    words = text.split()
    for i in range(len(words)):
        if words[i].startswith('#'):
            words[i] = 'talking about ' + ' '.join(word(words[i][1:]))
    return ' '.join(words)

In [10]:
replace_hashtag('I am #happyday')

NameError: name 'segment' is not defined

In [ ]:
text = " I am #happy    cde     cewbjk"
text.split()

In [15]:
txt = replace_hashtag('reasons2dothebirdmanhandrub')
segmented5 = segment_word(txt)
print(segmented5)


['reasons', '2dothebirdmanhandrub']


In [ ]:
import nltk
nltk.download('wordnet')
from nltk.stem import 	WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
text = ["studies", 'studying', 'cries', 'cry']
for w in text:
    print(w, " : ", wordnet_lemmatizer.lemmatize(w))

In [ ]:

def lemmatize_word(text: list)-> list:
    return [wordnet_lemmatizer.lemmatize(w) for w in text]

In [ ]:
lemmatize_word(["Don\'t", 'worry', "not" , 'be', 'happiness'])

In [ ]:
#segment 'I am sooooo #happy :D'

segmented = segment('iamhappy !!?')

In [ ]:
segmented

In [ ]:
lemmatize_word(segmented)

In [ ]:

change the emojis 
replace hashtag
segment
(penser a citer comment le dictionnaire de correction a ete fait)
replace more than 2 consecutive letters by 2
slang
error correction
lemmatize_word








In [1]:
from TextProc.Preprocessor import Preprocessor
from tqdm.notebook import tqdm as notebook_tqdm
from TextProc.preproc_configs import preproc_config_context, preproc_config_nocontext
import pandas as pd

/opt/homebrew/Caskroom/miniconda/base/envs/tensorflow-macos/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aliessonni/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
tweet_preprocessor = Preprocessor(preproc_config_context)

In [3]:
with open('Data/twitter-datasets/train_pos.txt', 'r') as f:
    train_pos = f.readlines()

In [4]:
df = pd.DataFrame(train_pos, columns=['text'])
# keep only the first 1000 tweets
df = df[:1000]

In [5]:
df.head()

,text
0,<user> i dunno justin read my mention or not ....
1,"because your logic is so dumb , i won't even c..."
2,""" <user> just put casper in a box ! "" looved t..."
3,<user> <user> thanks sir > > don't trip lil ma...
4,visiting my brother tmr is the bestest birthda...


In [6]:
preprocessed = tweet_preprocessor.preprocess(df)
# write the df to a txt file
preprocessed.to_csv('Data/preprocessed_train_pos_2.txt', sep='\t', index=False)

replace_emoticons


100%|██████████| 1000/1000 [00:00<00:00, 88106.38it/s]


make_two_consecutive


100%|██████████| 1000/1000 [00:00<00:00, 250451.07it/s]


replace_hashtag


...

100%|██████████| 1000/1000 [00:00<00:00, 1122371.96it/s]


correct_word


100%|██████████| 1000/1000 [00:00<00:00, 731990.23it/s]


list_to_string


100%|██████████| 1000/1000 [00:00<00:00, 2024277.99it/s]


In [3]:
tweets = [
    "<user> i dunno justin read my mention or not . only justin and god knows about that , but i hope you will follow me #believe 15",
    "because your logic is so dumb , i won't even crop out your name or your photo . tsk . <url>, #happyyy",
    "\" <user> just put casper in a box ! \" looved the battle ! #crakkbitch",
    "<user> <user> thanks sir > > don't trip lil mama ... just keep doin ya thang !",
    "<user> but seriously though .. it's called vanity fairest",
    "<user> <user> lol chloe :') ill teach yous to cook , you'll be pros by the end of it !",
    "<user> lol , im finna eat something ... chicken",
    "any questions for me ? just put preston down for a short nap and mikes not home for another hour , feel like answer q's ! ! make ' em original",
    "friday is payday & 4/20 ? ? hell yeah #reasons2dothebirdmanhandrub"
]

tweet_df = pd.DataFrame(tweets, columns=['text'])

In [4]:
preprocessed = tweet_preprocessor.preprocess(tweet_df)
for text in preprocessed['text']:
    print(text)
    print('------------------')

replace_emoticons
make_two_consecutive
replace_hashtag
remove_punctuation
string_to_list
correct_text
slang_to_english
correct_word
list_to_string
i I don't know justin read my mention or not only justin and god knows about that but i hope you will follow me talking about believe 15
------------------
because your logic is so dumb i won't even crop out your name or your photo tsk talking about happy
------------------
just put casper in a box loved the battle talking about Can't remember a okay bitch
------------------
thanks sir don't trip little mama pause just keep doing yeah thing
------------------
but seriously though it's called vanity fairest
------------------
laughing out loud chloe tears of joy ill teach yous to cook you'll be pros by the end of it
------------------
laughing out loud I am fitting to eat something pause chicken
------------------
any questions for me just put preston down for a short nap and mikes not home for another hour feel like answer q's make ' them or

In [14]:
for text in tweets:
    print(replace_hashtag(text))
    print('------------------')

<user> i dunno justin read my mention or not . only justin and god knows about that , but i hope you will follow me talking about believe 15
------------------
because your logic is so dumb , i won't even crop out your name or your photo . tsk . <url>
------------------
" <user> just put casper in a box ! " looved the battle ! talking about cr a kk bitch
------------------
<user> <user> thanks sir > > don't trip lil mama ... just keep doin ya thang !
------------------


In [12]:

wordninja.split('crackbitch')

['crack', 'bitch']

In [17]:
replace_hashtag('I am #happyday')

'I am talking about happy day'

In [10]:
from emoji import demojize
from googletrans import Translator
import re
from spellchecker import SpellChecker

def preprocess_tweets(tweets):
    processed_tweets = []

    # Initialize the translator and spell checker
    translator = Translator()
    spell = SpellChecker()

    for tweet in tweets:
        # Replace emoticons with a short description
        print('Demojizing...')
        tweet = demojize(tweet)

        print('Replacing #...')
        # Replace # with a description like 'talking about'
        tweet = re.sub(r'#', ' talking about ', tweet)

        print('Replacing @...')
        # Replace ... with a description like 'pause'
        tweet = re.sub(r'\.\.\.', ' pause ', tweet)

        print('Translating slang...')
        # Translate slang to English
        tweet = translator.translate(tweet).text

        print('Segmenting...')
        # Correct all spelling mistakes
        words = tweet.split()

        print('Correcting spelling...')
        corrected_words = [spell.correction(word) if spell.correction(word) != None else word for word in words]
        tweet = ' '.join(corrected_words)

        # Remove anything between < and >
        tweet = re.sub(r'<.*?>', '', tweet)

        # Remove punctuations
        tweet = re.sub(r'[^\w\s]', '', tweet)

        processed_tweets.append(tweet)

    return processed_tweets

# Input tweets
tweets = [
    "<user> i dunno justin read my mention or not . only justin and god knows about that , but i hope you will follow me #believe 15",
    "because your logic is so dumb , i won't even crop out your name or your photo . tsk . <url>, #happyyy",
    "\" <user> just put casper in a box ! \" looved the battle ! #crakkbitch",
    "<user> <user> thanks sir > > don't trip lil mama ... just keep doin ya thang !",
    "<user> but seriously though .. it's called vanity fairest",
    "<user> <user> lol chloe :') ill teach yous to cook , you'll be pros by the end of it !",
    "<user> lol , im finna eat something ... chicken",
    "any questions for me ? just put preston down for a short nap and mikes not home for another hour , feel like answer q's ! ! make ' em original",
    "friday is payday & 4/20 ? ? hell yeah #reasons2dothebirdmanhandrub"
]

# Preprocess tweets
preprocessed_tweets = preprocess_tweets(tweets)

# Display the preprocessed tweets
for i, tweet in enumerate(preprocessed_tweets, start=1):
    print(f"Processed Tweet {i}: {tweet}")


Demojizing...
Replacing #...
Replacing @...
Translating slang...
Segmenting...
Correcting spelling...
Demojizing...
Replacing #...
Replacing @...
Translating slang...
Segmenting...
Correcting spelling...
Demojizing...
Replacing #...
Replacing @...
Translating slang...
Segmenting...
Correcting spelling...
Demojizing...
Replacing #...
Replacing @...
Translating slang...
Segmenting...
Correcting spelling...
Demojizing...
Replacing #...
Replacing @...
Translating slang...
Segmenting...
Correcting spelling...
Demojizing...
Replacing #...
Replacing @...
Translating slang...
Segmenting...
Correcting spelling...
Demojizing...
Replacing #...
Replacing @...
Translating slang...
Segmenting...
Correcting spelling...
Demojizing...
Replacing #...
Replacing @...
Translating slang...
Segmenting...
Correcting spelling...
Demojizing...
Replacing #...
Replacing @...
Translating slang...
Segmenting...
Correcting spelling...
Processed Tweet 1: user i dunno justin read my mention or not  only justin and god

In [3]:
from spellchecker import SpellChecker

In [7]:
import spacy
from spellchecker import SpellChecker  # You may need to install this package using: pip install pyspellchecker

# Load the English NLP model from spaCy
nlp = spacy.load("en_core_web_sm")


# Initialize the spell checker
spell_checker = SpellChecker()

def split_and_correct_advanced(text):
    # Process the input text using spaCy
    doc = nlp(text)
    doc = str(doc).lower()
    print(type(doc))

    corrected_text = []

    for token in doc:
        # Check if the token is a word
        if token.is_alpha:
            # Check if the token is not a recognized word
            if not spell_checker.correction(token.text.lower()) == token.text.lower():
                # If not, consider it as a candidate for correction
                corrected_text.append(spell_checker.correction(token.text.lower()))
            else:
                corrected_text.append(token.text)
        else:
            corrected_text.append(token.text)

    return ' '.join(corrected_text)

# Example usage
input_text = "Thiss iss a testt sentennce withh sme mistakess and slanng."
corrected_text = split_and_correct_advanced(input_text)
print(corrected_text)


<class 'str'>


AttributeError: 'str' object has no attribute 'is_alpha'

In [11]:
with open('Data/twitter-datasets/train_pos.txt', 'r') as f:
    train_pos = f.readlines()

with open('Data/twitter-datasets/train_neg.txt', 'r') as f:
    train_neg = f.readlines()

import random
# sample 1% of the data at random
train_pos = random.sample(train_pos, int(len(train_pos) * 0.1))
train_neg = random.sample(train_neg, int(len(train_neg) * 0.1))

# write the sampled data to a txt file
with open('Data/twitter-datasets/train_pos_sample.txt', 'w') as f:
    f.writelines(train_pos)

with open('Data/twitter-datasets/train_neg_sample.txt', 'w') as f:
    f.writelines(train_neg)